In [9]:
import os
import networkx as nx
import boto3
import paramiko

# Run arguments
host_os = 'linux_mac' # or 'windows'
data_dir = '/home/rony/services/milestones_chains/data/'
data_file_name = 'EMS_DCMA_DD_23.08.graphml'
data_path = os.path.join(data_dir, data_file_name)
experiment = 'remote_test1'

## AWS
# IPs
resultsIP = '172.31.10.240'
serviceIP = '172.31.15.123'
# Compute instance
key_file_name = 'ds_eu_west2_2.pem'
if host_os=='windows':
    key_file_name = 'C:\\Users\\username\\.aws\\ds_eu_west2_2.pem'
else:
    key_file_name = '~/.aws/ds_eu_west2_2.pem'
INSTANCE_ID = 'i-0586e11281d4b02a2'
AWS_REGION = "eu-west-2"
EC2_RESOURCE = boto3.resource('ec2', region_name=AWS_REGION)
EC2_CLIENT = boto3.client('ec2', region_name=AWS_REGION)

# Storage bucket
profile_name = 'default'
data_bucket = 'programsdatabucket'
results_bucket = 'chainsresults'
session = boto3.session.Session(profile_name=profile_name)
s3_client = session.client('s3')
s3_resource = boto3.resource('s3')

In [3]:
# Start compute instance
instance = EC2_RESOURCE.Instance(INSTANCE_ID)
instance.start()
print(f'Starting EC2 instance: {instance.id}')
instance.wait_until_running()
print(f'EC2 instance "{instance.id}" has been started')

# Connect to instance to run process 
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
privkey = paramiko.RSAKey.from_private_key_file('ds_eu_west2_2.pem') # Works only with local pem file
ssh.connect(hostname=serviceIP, username='ubuntu', pkey=privkey)

Starting EC2 instance: i-0586e11281d4b02a2
EC2 instance "i-0586e11281d4b02a2" has been started


In [8]:
# Upload data to an S3 bucket
s3_client.upload_file(data_path, data_bucket, data_file_name)

#process_statement = "python3 remote_run_test.py"
ssh.exec_command('cd milestones_chains')
process_statement = 'python3 ./services/milestones_chains/run_pipelines.py'
process_statement = 'cd milestones_chains && python3 service.py'

stdin, stdout, stderr = ssh.exec_command(process_statement)
if stderr:
    print('Run attempt encountered error:\n', stderr.readlines())
elif stdout:
    print(stdout.readlines())
    # stdout = stdout.readlines()[0]
    # stdout = stdout.replace('\n', '')
    if stdout == 'completed':
        print(stdout)
        # Stop instance 
        response = EC2_CLIENT.stop_instances(InstanceIds=[INSTANCE_ID], DryRun=False)
        # Delete data from S3 bucket
        s3_resource.Object(data_bucket, data_file_name).delete()

Run attempt encountered error:
 ["python3: can't open file 'run_pipelines.py': [Errno 2] No such file or directory\n"]


In [10]:
response = EC2_CLIENT.stop_instances(InstanceIds=[INSTANCE_ID], DryRun=False)